# Topic Modelling

#### Evan Yathon

This notebook is intended to be run with papermill from the project root.

The purpose of this notebook is to process the review title and content to extract key phrases in each review.  Then key phrases will be used in a regression analysis to find out what is most important for a reviewer in recommending or not recommending the airline.

Topic modelling using LDA will be the tool of choice.

Usage:

`papermill src/ipynbs/topic_modelling.ipynb -p load_path data/cleaned_gw_reviews.csv`

In [1]:
#parameters section for Papermill

load_path = "../../data/cleaned_gw_reviews.csv"

In [ ]:
# loading packages

# utils
import pandas as pd
import re
import numpy as np

# NLP
import spacy
import gensim 
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim import models
import pyLDAvis.gensim

# display preference for ipynbs
%matplotlib inline

In [ ]:
# ensure the dates are parsed correctly with parse_dates argument
reviews = pd.read_csv(load_path, parse_dates = ["date_of_review", "date_flown"])

In [ ]:
reviews.head()

### Topic Modelling - Preprocessing

Prior to LDA, preprocessing of the text is necessary.  We need to:
- Tokenize each document (review)
- Minimum token length
- Remove stopwords
- Lemmatize
- Consider only specific parts of speech (nouns, verbs etc.)

Use [`spaCy`](https://spacy.io/) for this.

In [ ]:
# load the english spacy model
eng_nlp = spacy.load('en')

In [ ]:
# create a function to preprocess the reviews and review titles

def preprocess_text(text, min_token_len = 2, relevant_pos = ["NOUN", "VERB", "ADJ"]):
    """
    Given text, min_token_len, and relevant_pos carry out preprocessing of the text 
    and return a preprocessed string. 
    
    Keyword arguments:
    text (str): the text to be preprocessed
    min_token_len (int): min_token_length required
    relevant_pos (list): a list of relevant pos tags
    
    Returns: (str) the preprocessed text
    """
    # use several regex expressions to preprocess strange characters
    # or unwanted text
    
    # remove verified emoji
    text = re.sub(r"✅", "", text)
    
    # remove verified review and trip verified
    text = re.sub(r"Verified Review|Trip Verified", "", text)
    
    # remove anything that is not a word
    text = re.sub(r"[^\w]", " ", text)
    
    # replace multiple spaces with a single space
    text = re.sub(r"\s+", " ", text)
    
    # remove numbers
    text = re.sub(r"[0-9]+", "", text)
    
    # change all text to lowercase
    text = text.lower()
    
    # remove common terms found in topic modelling output
    # eg/ germanwings, flight
    # these words are not useful for this application as they
    # are common terms found in most reviews, and won't be indicative of 
    # things that influenced their recommendation
    terms = ["flight", "germanwing", "fly", "review"]
    text = re.sub("|".join(terms)," ", text)
    
    # utilize spaCy to tokenize and break text into lemmas/POS:
    # - remove stopwords
    # - reduce words to their lemmas
    # - only keep relevant parts of speech
    # - remove short words less than min_token_len
    
    # spacy will utilize
    doc = eng_nlp(text)
    
    processed_text = ""
    
    for token in doc:
        if not token.is_stop and len(token.text) >= min_token_len and token.pos_ in relevant_pos:
            processed_text += " " + token.lemma_
    
    return processed_text
    

Use the preprocessing function to process all review texts and review titles.

A good check will be to see if the review topics somewhat make sense with the title topics

In [ ]:
reviews["clean_review_text"] = reviews["review_text"].apply(preprocess_text, min_token_len = 3)
reviews["clean_title"] = reviews["title"].apply(preprocess_text, min_token_len = 3)

In [ ]:
reviews[["clean_review_text", "review_text", "clean_title", "title"]].head(10)

It looks mostly good, but I noticed that in the first review it is removing 'enough'.  This could be valuable in context, but I'll keep digging for now.  It's an issue since 'enough' is a stopword and is being removed.

### Topic Modelling: Dictionary and Document-Term Co-occurrence Matrix

In [ ]:
# change the cleaned reviews and cleaned titles to 
# an array, required by corpora.Dictionary
review_corpus = [doc.split() for doc in reviews["clean_review_text"].tolist()]
title_corpus = [doc.split() for doc in reviews["clean_title"].tolist()]

In [ ]:
# use gensim dictionary to map between words and integer ids
dct_review = corpora.Dictionary(review_corpus)
dct_title = corpora.Dictionary(title_corpus)

In [ ]:
# create the document-term co-occurrence matrices
doc_term_mat_review = [dct_review.doc2bow(doc) for doc in review_corpus]
doc_term_mat_title = [dct_title.doc2bow(doc) for doc in title_corpus]

### Topic Modelling:  LDA Topic Model

Number of topics was tuned using the visualizations below by examining the Intertopic distance map.  When there was no overlap and the topics were separated by a fair amount, the number of topics was chosen.

In [ ]:
# create the LDA models
# use a lower alpha, meaning that each document is representative of only a few topics
# do this so that each review doesn't contain similar topics and will have variation
# that we can use to figure out why the recommendation was a yes or no

lda_review = models.LdaModel(corpus = doc_term_mat_review, id2word = dct_review, num_topics = 4, alpha = 0.5, passes = 5)
lda_title = models.LdaModel(corpus = doc_term_mat_title, id2word = dct_title, num_topics = 3, alpha = 0.01, passes = 5)

In [ ]:
# view the topics and the words that describe them
lda_review.print_topics()

In [ ]:
# view the topics and the words that describe them
lda_title.print_topics()

### Topic Modelling: Visualization

In [ ]:
# view the reviews 
pyLDAvis.enable_notebook()
vis_reviews = pyLDAvis.gensim.prepare(lda_review, doc_term_mat_review, dct_review, sort_topics=False)
vis_reviews

In [ ]:
# view the titles
vis_titles = pyLDAvis.gensim.prepare(lda_title, doc_term_mat_title, dct_title, sort_topics=False)
vis_titles

### Topic Modelling: Assigning each Review a Topic

Create a handy function to help assign topics to the reviews dataframe

In [ ]:

def get_correct_topics(lda_doc_topics):
    """
    get_correct_topics uses an output LDA object to correctly place
    LDA outputs to their correct topic column number for use in a
    pandas dataframe
    
    Args:
        lda_doc_topics (TransformedCorpus): Document topic probabilities obtained through
                                            the .get_document_topics method of an LDA obj
    Returns:
        pandas dataframe of correctly assigned LDA topics
    """
    # placeholder array with same number of rows as the input 
    # corpus and number of topics for columns
    # initialize all zeros so that topics with missing probabilities
    # will be zero
    
    topic_prob_array = np.zeros(pd.DataFrame(lda_doc_topics).shape)

    for row_num, row in enumerate(lda_doc_topics):

        # assign each topic to the correct
        # topic[0] is the topic number
        # topic[1] is the corresponding probability
        for topic in row:
            topic_prob_array[row_num, topic[0]] = topic[1]

    topic_prob_df = pd.DataFrame(topic_prob_array)
    
    return topic_prob_df

Get both title and reviews dataframes, concatenate with `reviews`

In [ ]:
review_prob_df = get_correct_topics(lda_review.get_document_topics(doc_term_mat_review))
title_prob_df = get_correct_topics(lda_title.get_document_topics(doc_term_mat_title))

In [ ]:
title_prob_df.head()